In [ ]:
import random
import csv
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import math

In [ ]:
#Create type for minimization of the function
creator.create("FitnessMin",base.Fitness,weights=(-1.0,))

#Create type for store individual (in this example we use list) 
creator.create("Individual", list, fitness=creator.FitnessMin)


In [ ]:
CITY_NUMBER = 20
X_DIM = 100
Y_DIM = 100

#Create the location of the cities (map) for calculation distance in the fitness function
items = {}

for i in range(CITY_NUMBER):
        items[i] = (random.randint(0,X_DIM),random.randint(0,Y_DIM))
        
print(items)

In [ ]:
import numpy as np
# Define the code for plotting the city on the map
# Later, we need the function for plotting the obtained solution on the map (for evaluation)
import matplotlib.pyplot as plt 

fig, ax1 = plt.subplots()

# Set the size of the plot
fig.set_figwidth(10)
fig.set_figheight(10)

# Get the number of items for plotting 
number_of_items = len(items)
x = [items[i][0] for i in range(0,number_of_items) ]
y = [items[i][1] for i in range(0,number_of_items) ]

# Plot the points in the map (map: 100 x 100 pixels)
plt.scatter(x, y)

# Additionally, add a label for each point (further describe the order of the cities in the travel)
for i in range(0,number_of_items):
    ax1.annotate(i, (x[i]+1, y[i]+1))
    
plt.show()


In [ ]:
import csv
items = {}
i = 0
with open(f'TSP_{CITY_NUMBER}.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        items[i] = (int(row['x']), int(row['y']))
        i+=1

In [ ]:
#Create the class for rand cities with different order without repetition in an effective way for Deap framework
class RandCity:
    def __init__(self, city_number):
        self._city_number = city_number
        self._rand_tab = random.sample(range(self._city_number),self._city_number)
        self._i = 0
    def get_next_city_number(self):
        if self._i==self._city_number:
            self._i = 0
            self._rand_tab = random.sample(range(self._city_number),self._city_number)
        self._i += 1
        return self._rand_tab[self._i-1] 

In [ ]:
#Create toolbox for register needed functions
toolbox = base.Toolbox()
#Create object for get random cities without repeat
rc = RandCity(CITY_NUMBER) 

#Register function used for init content of the individual (for the benchmark function are the real values)
toolbox.register("attribute_int",rc.get_next_city_number)

#Register function used for init Population of the individuals
toolbox.register("individual", tools.initRepeat, 
creator.Individual, toolbox.attribute_int, n=CITY_NUMBER)

#Register function for create population of individuals
toolbox.register("population",tools.initRepeat, list, 
toolbox.individual)


In [ ]:
#Testing the registered function (look on the content of the individual)
ind1 = toolbox.individual()


In [ ]:
pop = toolbox.population(n=10)
print(pop)

In [ ]:
#Draw the exemplary path in the map
fig, ax1 = plt.subplots()
fig.set_figwidth(10)
fig.set_figheight(10)
size = len(items)

#Get list of the x and y values for plot path on the map
x = [items[i][0] for i in range(0,size) ]
y = [items[i][1] for i in range(0,size) ]
plt.scatter(x, y)


In [ ]:
#Get exemplary individual
individual = pop[0]
print(individual)


#Get the list of the x and y values for plot the path on the map using path encoded in the individual[0]
xp = [items[individual[i]][0] for i in range(0,size) ]
yp = [items[individual[i]][1] for i in range(0,size) ]
print(xp)
print(yp)
plt.plot(xp,yp)

# Add annotation (order) on the path
for i in range(0,size):
    ax1.annotate(i, (xp[i]+1, yp[i]+1))


In [ ]:
#Create function for display final results (the look of the path)
def plotPath(individual, items):
    fig, ax1 = plt.subplots()
    fig.set_figwidth(10)
    fig.set_figheight(10)
    size = len(items)
    x = [items[i][0] for i in range(0,size) ]
    y = [items[i][1] for i in range(0,size) ]
    
    plt.scatter(x, y)
    print(individual)
    xp = [items[individual[i]][0] for i in range(0,size) ]
    yp = [items[individual[i]][1] for i in range(0,size) ]
    print(xp)
    print(yp)
    plt.plot(xp,yp)
    
    for i in range(0,size):
        ax1.annotate(f'{i}:({xp[i]},{yp[i]})', (xp[i]+1, 
yp[i]+1))
    plt.show()


In [ ]:
plotPath(pop[1], items)

In [ ]:
def init_tau(CITY_NUMBER, start_pheromone_value):
    return np.full((CITY_NUMBER,CITY_NUMBER),start_pheromone_value)

In [ ]:
taus = init_tau(CITY_NUMBER,1)
print(taus)


In [ ]:
# Define the function for calc eta parameter between city1 and city2
# (Heuristic information between cities)
def calc_eta(city1, city2):
    if (city1 != city2):
        buf1 = items[city1][0] - items[city2][0]
        buf2 = items[city1][1] - items[city2][1]
        return 1.0/math.sqrt(buf1*buf1 + buf2*buf2)
    else:
        return 0.0


In [ ]:
# Define the function for calc etas values between all cities and 
# store in the matrix etas, the etas have the dimension [CITY_NUMBER x CITY_NUMBER]
# Function uses the items collection containing location of the cities
def calc_etas(items): 
    etas = np.array([[calc_eta(city1,city2) 
        for city1 in range(CITY_NUMBER)] 
            for city2 in range(CITY_NUMBER) ])
    return etas
etas = calc_etas(items)
# Uncoment please, if you need see etas
# print(etas)

In [ ]:
# Ant Colony System moving fun.(Gambardella, Dorigo 1996, 1997)
def move_ant_ACS(ant, taus, etas, alpha, beta, q0):
    feasible_cities = [i for i in range(CITY_NUMBER)]
    pos = 0 #start pos
    #Set the unvisited values in all pos
    for i in range(0,len(ant)):
        ant[i] = -1
    ant[pos] = random.randint(0,CITY_NUMBER-1) #start from random city
    
    #Until ant have unvisited city
    while len(feasible_cities) > 1:
        #remove last visited city from feasible cities
        feasible_cities.remove(ant[pos])
        #Probability of the selection for ACS system
        prob = np.array([ np.where(random.random() < q0 , 
         taus[pos][next_city] * etas[pos][next_city]**beta, 
         taus[pos][next_city]**alpha * etas[pos][next_city]**beta)
                        for next_city in feasible_cities ])
        #Roulette whell
        sum = np.sum(prob)
        if (sum>0):
            prob = prob/sum
        roulette = np.cumsum(prob)
        selectNextCity = np.argmax(roulette>random.random())
        
        #Go to the next city accoring to the roulette whell selection
        pos += 1
        ant[pos] = feasible_cities[selectNextCity]
    return ant


In [ ]:
# EAS - Elitist Ant System (Dorigo 1992)
def update_tau_EAS(pop, tau, ro, e):
    # Get fitness
    fit = np.array([ant.fitness.values for ant in pop])
    # Calc Delta tau for all ants
    delta_tau_ant = 1.0/fit
    
    # Calc Delta tau
    delta_tau = np.full((CITY_NUMBER,CITY_NUMBER),0.0)
    for ai in range(0,len(pop)):
        ant = pop[ai]
        for city1, city2 in zip(ant[::1],ant[1::1]):
            delta_tau[city1][city2] += delta_tau_ant[ai]
            
    #Additional update by the best ant (always is on the pos 0)
    ant = pop[0]
    for city1, city2 in zip(ant[::1],ant[1::1]):
            delta_tau[city1][city2] += delta_tau_ant[0] * e
    tau = (1.0 - ro) * tau + delta_tau
    return tau

In [ ]:
# Ant Colony System (Gambardella, Dorigo 1996, 1997)
def update_global_ACS(pop, it_best_ant, tau, ro):
    # Calc delta for the iteration best ant
    delta_it_best = 1.0/it_best_ant.fitness.values[0]
    # Updtate only the iteration best ant
    for city1, city2 in zip(it_best_ant[::1],it_best_ant[1::1]):
        tau[city1][city2] = (1-ro) * tau[city1][city2] + ro * delta_it_best
    return tau
# Ant Colony System (ACS) - (Gambardella, Dorigo 1996, 1997)
def update_local_ACS(pop, tau, epsilon, tau_0):
    for ai in range(0,len(pop)):
        ant = pop[ai]
        for city1, city2 in zip(ant[::1],ant[1::1]):
            tau[city1][city2] = (1.0 - epsilon) * tau[city1][city2] + epsilon * tau_0
    return tau


In [ ]:
# MMA - Max-Min Ant System (Stutzle, Hoos 1996-2001)
def update_global_MMA(pop, it_best_ant, tau, ro, tau_min, tau_max):
    # Calc delta for the iteration best ant
    delta_it_best = 1.0/it_best_ant.fitness.values[0]
    
    # Updtate only the iteration best ant
    for city1, city2 in zip(it_best_ant[::1],it_best_ant[1::1]):
            tau[city1][city2] = (1-ro) * tau[city1][city2] + delta_it_best
            # check constrains
            if (tau[city1][city2] < tau_min):
                tau[city1][city2] = tau_min
            if (tau[city1][city2] > tau_max):
                tau[city1][city2] = tau_max
    return tau


In [ ]:
toolbox.register("init_pheromones",init_tau, start_pheromone_value = 
0.01)
toolbox.register("calc_heuristic_information",calc_etas)
toolbox.register("move",move_ant_ACS,q0 = 0.4)
toolbox.register("update_tau",update_tau_EAS)
toolbox.register("evaluate",fitness_tsp)
toolbox.register("update_tau", update_tau_EAS, ro = 0.01, e = 2.0  )


In [ ]:
def ANT_ALG2(alpha,beta,NGEN,N,stats): 
    pop = toolbox.population(n=N) #Create a new pop(1)
    logbook = tools.Logbook() 
    etas = toolbox.calc_heuristic_information(items) #heuristic information
    taus = toolbox.init_pheromones(CITY_NUMBER) #pheromones
    # Evaluate the entire population #(2)
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    #Find the best ant and remember it
    best_ant = toolbox.clone(pop[0])
    for ant in pop:
        if best_ant.fitness.values > ant.fitness.values:
            best_ant = toolbox.clone(ant)
    #Create the iteration best ant
    it_best_ant = toolbox.clone(best_ant)
    # First time update the pheromone trail
    taus = toolbox.update_tau(pop,taus)
    #taus = toolbox.update_tau_best(pop,it_best_ant,taus)
    #Loop over generations
    for g in range(NGEN):
        # Apply moving of the ant
        for ai in range(0,N):
            pop[ai] = toolbox.move(pop[ai],taus, etas, alpha,beta)
        # Evaluate the individuals with an invalid fitness #(2)
        fitnesses = map(toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit
        # Find the iteration best ant
        it_best_ant = toolbox.clone(pop[0])
        # Update the best ant (best - min)
        for ant in pop:
            # global best
            if best_ant.fitness.values > ant.fitness.values:
                best_ant = toolbox.clone(ant);
            # iteration best
            if it_best_ant.fitness.values > ant.fitness.values:
                it_best_ant = toolbox.clone(ant)
                # And remeber as the first (global best)
        pop[0] = toolbox.clone(best_ant); 
        # Update the pheromone trail
        taus = toolbox.update_tau(pop,taus)
        #taus = toolbox.update_tau_best(pop,it_best_ant,taus)
        record = stats.compile(pop) #record the statistics
        logbook.record(gen=g, evals=len(pop), **record)
    print(f'The final value for taus is {taus}')
    
    return pop, logbook


In [ ]:
#Build statistics for the algorithm
import numpy
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg",numpy.mean)
stats.register("std",numpy.std)
stats.register("min",numpy.min)
stats.register("max",numpy.max)


In [ ]:
# Run the algoritm
pop, logbook = ANT_ALG2(0.5,2,400,100,stats)
gen, avg, std, Min, Max = logbook.select('gen','avg','std','min','max')

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()
fig.set_figwidth(20)
fig.set_figheight(10)
line1 = ax1.plot(gen, Min, "r-")
ax1.legend()
ax2 = ax1.twinx()
line2 = ax2.plot(gen, avg, "b-")

ax1.set_xlabel("Generations")
ax1.set_ylabel("Fitness", color="b")

plt.show()
print(Min)

In [ ]:
evals = [ fitness_tsp(p)[0] for p in pop]
print(evals)


In [ ]:
evals = [ fitness_tsp(p)[0] for p in pop]
print(evals)
best = min(evals)
print(best)
pos = evals.index(best)
print(pos)
plotPath(pop[pos],items)


In [ ]:
# Create function for display finall taus on the pheromone map
# Important function for evaluationg, how correct are the parameters: alpha and beta
def plotPheromoneTrails(items, taus):
    fig, ax1 = plt.subplots()
    fig.set_figwidth(10)
    fig.set_figheight(10)
    size = len(items)
    x = [items[i][0] for i in range(0,size) ]
    y = [items[i][1] for i in range(0,size) ]
    
    #The city location
    plt.scatter(x, y)
    
    max_value = np.max(taus)
    taus_Norm = taus/max_value
    # Div the color to the 3 components (RGB) for better look
    # 0-0.33 - small pheromone value (wrong way)
    # 0.33-0.66 - average pheromone value
    # 0.66-1.0 - bigger pheromone value (better way, the way wisited by the most of the colony)

    for p1 in items:
        for p2 in items:
            if (p1!=p2):
                if (taus_Norm[p1][p2]<0.33):
                    plt.plot([items[p1][0], 
items[p2][0]],[items[p1][1],items[p2][1]],color=(taus_Norm[p1][p2],0,0)
)
                elif (taus_Norm[p1][p2]<0.66):
                    plt.plot([items[p1][0], 
items[p2][0]],[items[p1][1],items[p2][1]],color=(0,taus_Norm[p1][p2],0)
)
                else:
                    plt.plot([items[p1][0], 
items[p2][0]],[items[p1][1],items[p2][1]],color=(0,0,taus_Norm[p1][p2])
)
    plt.show()


In [ ]:
plotPheromoneTrails(items,taus)


In [ ]:
etas = toolbox.calc_heuristic_information(items)
plotPheromoneTrails(items,taus)


In [ ]:
'''
We propose a new model of ant colony optimization (ACO) to solve
the traveling salesman problem (TSP) by introducing ants with memory
into the ant colony system (ACS). In the new ant system, the ants 
can remember and make use of the best-so-far solution, 
so that the algorithm is able to converge into at least
a near-optimum solution quickly. We have tested the algorithm 
in 3 representational TSP instances and compared the results
with the original ACS algorithm. According to the result 
we make amelioration to the new ant model and test it again. 
The simulations show that the amended ants with memory improve 
the converge speed and can find better solutions compared 
to the original ants.
'''